# Import Dependencies

In [42]:
import numpy as np
import pandas as pd
import collections as c
import pickle
import gc

In [29]:
# define the data path
datasource = "data/"

# Import Prices

In [30]:
df = pd.read_excel(datasource+"Rosies500InExcel.xlsx", sheet_name = "Price")

In [31]:
df.sort_values(by=['date'], inplace = True)
df.drop(columns=["e"], inplace = True)

In [32]:
df.head()

,PERMNO,date,TICKER,Company Name,CUSIP,Price,Share Volume
0,10078,19930129,SUNW,SUN MICROSYSTEMS INC,86681020,39.000,421287.0
128317,72726,19930129,STBK,STATE STREET BOSTON CORP,85747710,44.500,38317.0
128629,73139,19930129,STRY,STRYKER CORP,86366710,36.000,75237.0
26658,17144,19930129,GIS,GENERAL MILLS INC,37033410,67.500,53731.0
128941,73940,19930129,SBL,SYMBOL TECHNOLOGIES INC,87150810,14.375,46883.0


In [33]:
# Create lookup dicts
cusip_to_company = c.defaultdict(set)
company_to_cusip = {}

for _, row in df.iterrows():
    cusip = row["CUSIP"]
    name = row["Company Name"]
    cusip_to_company[cusip].add(name)
    company_to_cusip[name]=cusip

In [34]:
print(cusip_to_company['86681020'])
print(company_to_cusip['APPLE INC'])

{'SUN MICROSYSTEMS INC'}
03783310


In [35]:
# creating list of dicts for new df
dates = df["date"].unique()
data = []
for i in dates:
    temp_df = df[df["date"]==i]
    mydict = {}
    for _, row in temp_df.iterrows():
        price = row["Price"]
        cusip = row["CUSIP"]
        mydict[cusip] = price
    data.append(mydict)

In [36]:
price_df = pd.DataFrame(data, index = dates)
price_df.head()

,00088630,00101J10,00105510,00120410,00130H10,00154710,00195750,00206R10,00209A10,00282410,...,G9782210,G9829410,H1467J10,H2717810,H2906T10,H8498910,H8817H10,N5374510,N5946510,V7780T10
19930129,46.750,NaN,33.625,38.500,27.25,NaN,52.875,68.875,NaN,27.500,...,45.00,NaN,NaN,29.750,NaN,NaN,NaN,NaN,35.875,NaN
19930226,41.500,NaN,35.000,40.875,28.75,NaN,55.875,74.500,NaN,26.125,...,41.00,NaN,NaN,30.000,NaN,NaN,NaN,NaN,30.500,NaN
19930331,44.125,NaN,36.000,42.000,30.75,NaN,56.750,78.125,NaN,25.750,...,44.75,NaN,29.000,30.375,NaN,NaN,NaN,NaN,29.000,NaN
19930430,41.250,NaN,37.500,40.375,31.50,NaN,55.875,77.250,NaN,27.875,...,44.25,NaN,28.625,27.500,NaN,NaN,NaN,NaN,27.125,18.375
19930528,55.000,NaN,39.500,39.000,30.25,NaN,61.500,38.375,NaN,27.125,...,44.75,NaN,28.625,27.625,NaN,NaN,22.125,NaN,29.000,17.375


In [37]:
# Cleaning up price_df

# Taking 3 month price average (i.e. quarterly price)
series = []
for i in range(104):
    series.append(price_df[i:i+3].mean())

# Concatenating the series and transposing the df
price_df = pd.concat(series, axis=1).transpose()

# years
years = []
for i in dates:
    years.append(int(str(i)[:4]))
    
# quarters
quarters = set()
count = 1
for i in years:
    if count == 5:
        count = 1
    quarters.add(str(i)+"Q"+str(count))
    count += 1
quarters = list(quarters)
quarters.sort()

# setting quarters as index
price_df.index = quarters
price_df = price_df.transpose()

In [38]:
price_df.head()

,1993Q1,1993Q2,1993Q3,1993Q4,1994Q1,1994Q2,1994Q3,1994Q4,1995Q1,1995Q2,...,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4
00088630,44.125000,42.291667,46.791667,42.500000,38.666667,32.416667,35.000000,37.250000,36.666667,35.541667,...,17.625000,14.604167,11.395833,9.045000,7.896667,7.263333,6.396667,5.293333,4.256667,4.136667
00101J10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00105510,34.875000,36.166667,37.666667,35.125000,32.500000,30.333333,31.625000,32.250000,30.250000,29.000000,...,67.180833,63.775833,48.893333,39.833333,30.590000,31.906667,31.166667,29.530000,28.033333,26.326667
00120410,40.458333,41.083333,40.458333,39.833333,39.791667,39.500000,38.666667,37.500000,36.875000,37.291667,...,21.687500,21.397500,21.346667,22.130000,22.753333,23.366667,23.750000,23.016667,21.756667,20.640000
00130H10,28.916667,30.333333,30.833333,30.666667,30.750000,31.083333,31.708333,32.041667,32.375000,33.333333,...,54.960000,55.658333,53.853333,50.533333,47.676667,45.373333,42.250000,38.156667,28.080000,19.930000


In [41]:
# export to excel file
price_df.to_excel(datasource+"price_df.xlsx")

# saving lookup dicts
pickle.dump(cusip_to_company, open(datasource+"cusip_to_company.pkl", "wb"))
pickle.dump(company_to_cusip, open(datasource+"company_to_cusip.pkl", "wb"))

In [43]:
del df
del price_df
gc.collect()

96478

# Import Features

In [44]:
df2 = pd.read_excel(datasource+"Rosies500InExcel.xlsx", sheet_name = "Cleaned Features")

In [45]:
df2.sort_values(by=['datadate'],inplace=True)
columns_to_drop = ["gvkey", "datadate", "fyearq", "fqtr", "fyr", "indfmt", \
                   "consol", "popsrc", "datafmt", "tic", "curcdq", "datacqtr"]
df2.drop(columns = columns_to_drop, inplace=True)

In [46]:
df2.shape

(70976, 71)

In [47]:
df2.head()

,cusip,conm,datafqtr,acoq,actq,aoq,apq,atq,ceqq,cheq,...,tstkq,txpq,txtq,wcapq,xidoq,xintq,xiq,xoprq,Unnamed: 81,cik
0,000886309,ADC TELECOMMUNICATIONS INC,1993Q1,8.6,114.607,66.529,9.440,238.483,190.883,20.878,...,0.000,NaN,3.090,76.393,0.000,NaN,0.000,64.491,NaN,61478.0
42336,089302103,BIG LOTS INC,1992Q4,21.909,261.620,2.491,77.644,390.942,209.459,35.254,...,0.000,11.963,14.631,142.305,0.000,1.408,0.000,260.710,NaN,768835.0
42441,617446448,MORGAN STANLEY,1992Q4,0,NaN,191.066,10479.699,80353.188,2929.453,25339.598,...,139.433,0.000,52.945,NaN,0.000,1099.783,0.000,500.882,NaN,895421.0
18684,440452100,HORMEL FOODS CORP,1993Q1,7.337,584.893,90.691,80.020,891.444,654.832,224.953,...,0.486,12.430,10.477,399.109,-127.529,0.308,-127.529,647.858,NaN,48465.0
43760,592907109,MEYER (FRED) INC,1992Q4,57.496,530.173,6.720,259.230,1079.103,450.128,31.884,...,3.896,15.418,15.170,185.257,0.000,NaN,0.000,696.155,NaN,1043273.0


In [ ]:
# export to excel file
feature_df.to_excel(datasource+"feature_df.xlsx")